In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time

from PIL import Image

from keras.preprocessing import sequence
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.densenet import DenseNet121

from keras.layers import Input
from keras.layers import Dropout
from keras.layers.core import Dense
from keras.models import Model
from keras import backend as K
#print(dir(keras.applications.densenet.DenseNet121))

from time import time
import pickle
from pickle import dump, load
import glob
import os
from random import shuffle
import importlib
from configparser import ConfigParser

Using TensorFlow backend.


In [355]:
!pip install pydot

### Loading report sentences from pickle file

In [2]:
first_sent = load(open("KalyanPickle/para.pickle", "rb"))
print('Photos: para.pickle=%d' % len(first_sent))

Photos: para.pickle=3668


In [3]:
first_sent

{'CXR1_1_IM-0001-3001': ['startseq the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. endseq'],
 'CXR10_IM-0002-1001': ['startseq the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax. stable calcified granuloma within the right upper lung. no acute bone abnormality. . endseq'],
 'CXR100_IM-0002-1001': ['startseq both lungs are clear and expanded. heart and mediastinum normal. endseq'],
 'CXR1000_IM-0003-1001': ['startseq there is xxxx increased opacity within the right upper lobe with possible mass and associated area of atelectasis or focal consolidation. the cardiac silhouette is within normal limits. xxxx opacity in the left midlung overlying the posterior left 5th rib may represent focal airsp

### Loading indication sentences from pickle file

In [4]:
indication_sent = load(open("KalyanPickle/indication_keys.pickle", "rb"))
print('Photos: indication=%d' % len(indication_sent))

Photos: indication=3668


In [5]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
    """
        **Convert descriptions to  vocabulary**
        
        This function will create vocabulary using descriptions
        
        :param descriptions: image to report key-value pairs
        :type descriptions: dict
        :return: set of unique words of all descriptions - vocabulary
        
        - Example::

            descriptions - {'CXR1_1_IM-0001-3001': ['startseq the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. endseq'],
 'CXR10_IM-0002-1001': ['startseq the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax. stable calcified granuloma within the right upper lung. no acute bone abnormality. . endseq']
                            }
                            
            to_vocabulary(descriptions)
            
        - Expected Success Response::

            {'startseq',
            'the',
            'cardiac',
            ...
            'endseq'
            }
    
    """
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(first_sent)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 2311


In [6]:
all_indications = set()
for key in indication_sent.keys():
    all_indications.update(indication_sent[key].split())
indicationVocab = all_indications
print('Original Vocabulary Size: %d' % len(indicationVocab))

Original Vocabulary Size: 1572


### Loading images into train, validation and test datasets

In [7]:
# Below path contains all the images
images = 'NLMCXR_png/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.png')

In [8]:
train_img_paths = img[:int(len(img)*0.70)]
valid_img_paths = img[int(len(img)*0.70): int(len(img)*0.70 + len(img)*0.20)]
test_img_paths =  img[int(len(img)*0.70 + len(img)*0.20):]

In [9]:
print("No of images for training: ", len(train_img_paths))
print("No of images for validating: ", len(valid_img_paths))
print("No of images for testing: ", len(test_img_paths))

No of images for training:  5229
No of images for validating:  1494
No of images for testing:  747


In [10]:
def get_train_valid_test_data(img):
    """
        **Splits images into test and train sets of ratio 90:10**
        
        This function will create test and train sets of ratio 90:10 with given image sets
        
        :param img: image to split
        :type img: array
        :return: set of images for training and testing
        
        - Example::

            img - ['NLMCXR_png\\CXR1000_IM-0003-1001.png',
                   'NLMCXR_png\\CXR1000_IM-0003-2001.png',........
                   'NLMCXR_png\\CXR1000_IM-0003-3001.png']
                            
            train_imgs, valid_imgs = get_train_valid_test_data(img)
            
        - Expected Success Response::

            train_imgs - ['NLMCXR_png\\CXR1000_IM-0003-1001.png',
                          'NLMCXR_png\\CXR1000_IM-0003-2001.png',
                          'NLMCXR_png\\CXR1000_IM-0003-3001.png',......
                          'NLMCXR_png\\CXR1001_IM-0004-1001.png']
            valid_imgs - ['NLMCXR_png\\CXR614_IM-2200-1001.png',
                          'NLMCXR_png\\CXR614_IM-2200-4001.png',
                          'NLMCXR_png\\CXR615_IM-2200-1001.png',
                          'NLMCXR_png\\CXR615_IM-2200-1002.png',.......
                          'NLMCXR_png\\CXR616_IM-2200-1001.png']
    
    """
    img = [item.split("/")[-1] for item in img]
    train_imgs = img[:int(len(img)*0.70)]
    valid_imgs = img[int(len(img)*0.70): int(len(img)*0.70 + len(img)*0.20)]
    test_imgs =  img[int(len(img)*0.70 + len(img)*0.20):]
    return train_imgs, valid_imgs, test_imgs

In [11]:
train_imgs, valid_imgs, test_imgs = get_train_valid_test_data(img)

### Loading chexnet model for generation of image features

In [12]:
weights_path = "PavanPickle/best_weights.h5"

In [13]:
class ModelFactory:
    """
    Model facotry for Keras default models
    """

    def __init__(self):
        self.models_ = dict(
            VGG16=dict(
                input_shape=(224, 224, 3),
                module_name="vgg16",
                last_conv_layer="block5_conv3",
            ),
            VGG19=dict(
                input_shape=(224, 224, 3),
                module_name="vgg19",
                last_conv_layer="block5_conv4",
            ),
            DenseNet121=dict(
                input_shape=(224, 224, 3),
                module_name="densenet",
                last_conv_layer="bn",
            ),
            ResNet50=dict(
                input_shape=(224, 224, 3),
                module_name="resnet50",
                last_conv_layer="activation_49",
            ),
            InceptionV3=dict(
                input_shape=(299, 299, 3),
                module_name="inception_v3",
                last_conv_layer="mixed10",
            ),
            InceptionResNetV2=dict(
                input_shape=(299, 299, 3),
                module_name="inception_resnet_v2",
                last_conv_layer="conv_7b_ac",
            ),
            NASNetMobile=dict(
                input_shape=(224, 224, 3),
                module_name="nasnet",
                last_conv_layer="activation_188",
            ),
                        NASNetLarge=dict(
                input_shape=(331, 331, 3),
                module_name="nasnet",
                last_conv_layer="activation_260",
            ),
        )

    def get_last_conv_layer(self, model_name):
        return self.models_[model_name]["last_conv_layer"]

    def get_input_size(self, model_name):
        return self.models_[model_name]["input_shape"][:2]

    def get_model(self, model_name="DenseNet121", use_base_weights=True,
                  weights_path=None, input_shape=None):
        """
        **Provides the requested CNN pre trained model**
        
        This function provides the requested CNN pre trained model
        
        :param model_name: pre trained model name
        :type model_name: string
        
        :param use_base_weights: Indicates to use the base weights or not
        :type use_base_weights: bool
        
        :param weights_path: location path where the weigths are present
        :type weights_path: string
        
        :param input_shape: shape of the input image to the pretrained model
        :type input_shape: array
        
        :return: Pretrained CNN Model
        
        - Example::
        
            weights_path = "PavanPickle/best_weights.h5"
            
            model = model_factory.get_model(use_base_weights=False,weights_path=weights_path)
            
        - Expected Success Response::
            model
    
        """
        
        if use_base_weights is True:
            base_weights = "imagenet"
        else:
            base_weights = None

        base_model_class = getattr(
            importlib.import_module(
                f"keras.applications.{self.models_[model_name]['module_name']}"
            ),
            model_name)

        if input_shape is None:
            input_shape = self.models_[model_name]["input_shape"]

        img_input = Input(shape=input_shape)

        base_model = base_model_class(
            include_top=False,
            input_tensor=img_input,
            input_shape=input_shape,
            weights=base_weights,
            pooling="avg")
        x = base_model.output
        predictions = Dense(14, activation="sigmoid", name="predictions")(x)
        model = Model(inputs=img_input, outputs=predictions)

        if weights_path == "":
            weights_path = None
        

        if weights_path is not None:
            print(f"load model weights_path: {weights_path}")
            model.load_weights(weights_path)
        return model

In [14]:
model_factory = ModelFactory()
model = model_factory.get_model(
    use_base_weights=False,
    weights_path=weights_path)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
load model weights_path: PavanPickle/best_weights.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]  

In [15]:
def load_image(image_path):
        """
        **Loads the Image from the provided path**
        
        This function will loads the image from the provided path
        
        :param image_path: path of the image files
        :type image_path: string
        :return: Image
        :type Image: array
        
        - Example::

            img - ['NLMCXR_png\\CXR1000_IM-0003-1001.png',
                   'NLMCXR_png\\CXR1000_IM-0003-2001.png',........
                   'NLMCXR_png\\CXR1000_IM-0003-3001.png']
                            
            imgs = load_image(img)
            
        - Expected Success Response::

            image
    
        """
        image = Image.open(image_path)
        image_array = np.asarray(image.convert("RGB"))
        image_array = image_array / 255.
        image_array = np.resize(image_array, (1,224,224,3))
        return image_array

In [16]:
def get_img_encoding(img):
    """
        **Provides the Image encoding for the given image**
        
        This function will provides the Image encoding for the given image
        
        :param img: Imput image to get the encoding
        :type descriptions: array
        :return: Image encoding
        :type: array
        
        - Example::

            sample_img_encoding = get_img_encoding(img[0])
            sample_img_encoding.shape
            
        - Expected Success Response::

            (1024,)
    
    """
    #img = load_image(train_img_paths[0])
    img = load_image(img)
    get_last_but_one_layer_output = K.function([model.layers[0].input],
                                        [model.layers[-2].output])
    layer_output = get_last_but_one_layer_output([img])
    return layer_output[0].squeeze()

In [17]:
sample_img_encoding = get_img_encoding(img[0])

In [18]:
sample_img_encoding.shape

(1024,)

### Loading generated train and validation features

In [19]:
train_features = load(open("PavanPickle/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=5229


In [20]:
train_features

{'CXR1000_IM-0003-1001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR1000_IM-0003-2001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR1000_IM-0003-3001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR1001_IM-0004-1001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR1001_IM-0004-1002.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR1002_IM-0004-1001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR1002_IM-0004-2001.png': array([[6.4905726

In [21]:
encoding_test = load(open("PavanPickle/encoded_test_images.pkl", "rb"))
print('Photos: train=%d' % len(encoding_test))

Photos: train=1494


In [22]:
encoding_test

{'CXR3533_IM-1726-2001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR3534_IM-1727-1001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR3534_IM-1727-2001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR3535_IM-1728-0001-0001.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR3535_IM-1728-0001-0002.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR3536_IM-1729-4004.png': array([[6.4905726e-05, 1.3562142e-03, 1.3405833e-03, ..., 7.7307308e-01,
         8.1688350e-01, 6.3264632e-01]], dtype=float32),
 'CXR3536_IM-1729-5001.png': array([

In [23]:
# Create a list of all the training captions
all_captions = []
for key, val in first_sent.items():
    for cap in val:
        all_captions.append(cap)
len(all_captions)

3668

In [24]:
all_captions

['startseq the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. endseq',
 'startseq the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax. stable calcified granuloma within the right upper lung. no acute bone abnormality. . endseq',
 'startseq both lungs are clear and expanded. heart and mediastinum normal. endseq',
 'startseq there is xxxx increased opacity within the right upper lobe with possible mass and associated area of atelectasis or focal consolidation. the cardiac silhouette is within normal limits. xxxx opacity in the left midlung overlying the posterior left 5th rib may represent focal airspace disease. no pleural effusion or pneumothorax. no acute bone abnormality. endseq',
 'startseq in

In [25]:
# Create a list of all the training indications
all_indications = []
for key, val in indication_sent.items():
    all_indications.append(val)
len(all_indications)

3668

In [26]:
len(indication_sent)

3668

### Getting clean descriptions out of reports and indications

In [27]:
# load clean descriptions into memory
def load_clean_descriptions(desc, dataset):
    """
        **Loads the findings which are all available for the trainging images**
        
        This function will load the findings which are all available for the trainging images
        
        :param desc: report findings
        :type desc: dict
        :param dataset: trainning images
        :type desc: list
        :return: findings available for the trainning images
        :type: dict
        
        - Example::
                            
            train_descriptions = load_clean_descriptions(first_sent, dataset)
            print('Descriptions: train=%d' % len(train_descriptions))
            
        - Expected Success Response::

            Descriptions: train=3218
    
    """
    descriptions = dict()
    for key, val in desc.items():
        image_id, image_desc = key, val
        image_id = image_id+'.png'
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = ''.join(image_desc)
            # store
            descriptions[image_id].append(desc)
    return descriptions

In [28]:
dataset = list()
for img in train_imgs:
    identifier = img[len(images):]
    dataset.append(identifier)

In [29]:
# descriptions
train_descriptions = load_clean_descriptions(first_sent, dataset)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=2559


In [30]:
# descriptions
train_indications = load_clean_descriptions(indication_sent, dataset)
print('Descriptions: indication=%d' % len(train_indications))

Descriptions: indication=2559


In [31]:
train_descriptions['CXR1_1_IM-0001-3001.png'][0][9:-7]

'the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax.'

In [32]:
#cleaned_train_descr = dict()
i=0
for key in train_descriptions.keys():
    print(train_descriptions[key][0][9:-7])
    i+=1
    if i==10:
        break

the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax.
the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax. stable calcified granuloma within the right upper lung. no acute bone abnormality. .
both lungs are clear and expanded. heart and mediastinum normal.
there is xxxx increased opacity within the right upper lobe with possible mass and associated area of atelectasis or focal consolidation. the cardiac silhouette is within normal limits. xxxx opacity in the left midlung overlying the posterior left 5th rib may represent focal airspace disease. no pleural effusion or pneumothorax. no acute bone abnormality.
interstitial markings are diffusely prominent throughout both lungs. heart size is normal. pu

In [33]:
cleaned_train_descr = dict()
for key in train_descriptions.keys():
    cleaned_train_descr[key]=list()
    cleaned_train_descr[key].append('startseq. '+ train_descriptions[key][0][9:-7].strip('.') +'. endseq')

In [34]:
cleaned_train_descr['CXR1_1_IM-0001-3001.png']

['startseq. the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. endseq']

In [35]:
# Create a list of all the training captions
all_train_captions = []
for key, val in cleaned_train_descr.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

2559

In [36]:
# Create a list of all the training captions
all_train_indications = []
for key, val in train_indications.items():
    for cap in val:
        all_train_indications.append(cap)
len(all_train_indications)

2559

In [37]:
# Consider only words which occur at least 10 times in the corpus
sent_count_threshold = 1
sent_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for s in sent.split('. '):
        sent_counts[s] = sent_counts.get(s, 0) + 1

sent_vocab = [s.strip('.') for s in sent_counts if sent_counts[s] >= sent_count_threshold and s!='.']
print('preprocessed sentences %d -> %d' % (len(sent_counts), len(sent_vocab)))

preprocessed sentences 4628 -> 4628


In [38]:
sent_vocab[:10]

['startseq',
 'the cardiac silhouette and mediastinum size are within normal limits',
 'there is no pulmonary edema',
 'there is no focal consolidation',
 'there are no xxxx of a pleural effusion',
 'there is no evidence of pneumothorax',
 'endseq',
 'the cardiomediastinal silhouette is within normal limits for size and contour',
 'the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax',
 'stable calcified granuloma within the right upper lung']

In [39]:
# Consider only words which occur at least 10 times in the corpus
indication_count_threshold = 1
indication_counts = {}
nsents = 0
for sent in all_train_indications:
    nsents += 1
    for w in sent.split(' '):
        indication_counts[w] = indication_counts.get(w, 0) + 1

indicationVocab = [w for w in indication_counts if indication_counts[w] >= indication_count_threshold]
print('preprocessed words %d -> %d' % (len(indication_counts), len(indicationVocab)))

preprocessed words 1291 -> 1291


### Creating sentence vocabulary out of report sentences & word vocabulary out of indications

In [40]:
ixtosent = {}
senttoix = {}

ix = 1
for s in sent_vocab:
    senttoix[s] = ix
    ixtosent[ix] = s
    ix += 1

In [41]:
sent_vocab_size = len(ixtosent) + 1 # one for appended 0's
sent_vocab_size

4629

In [42]:
ixtoindicationWord = {}
indicationWordtoix = {}

ix = 1
for w in indicationVocab:
    indicationWordtoix[w] = ix
    ixtoindicationWord[ix] = w
    ix += 1

In [43]:
IndicationVocab_size = len(ixtoindicationWord) + 1 # one for appended 0's
IndicationVocab_size

1292

In [44]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    """
        **Extracts all the findings from the input dictionary of findings**
        
        This function will create list of findings
        
        :param descriptions: image to report key-value pairs
        :type descriptions: dict
        :return: list of findings in report
        
        - Example::

            descriptions - {'CXR1_1_IM-0001-3001': ['startseq the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. endseq'],
 'CXR10_IM-0002-1001': ['startseq the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax. stable calcified granuloma within the right upper lung. no acute bone abnormality. . endseq']
                            }
                            
            lines = to_lines(descriptions)
            print(len(lines))
            
        - Expected Success Response::

            172
    
    """
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
    """
        **Provides the Maximum length of findings available in trainning set**
        
        This function will provide the Maximum length of findings available in trainning set
        
        :param descriptions: image to report key-value pairs
        :type descriptions: dict
        :return: list of findings in report
        :type : int
        - Example::

            train_descriptions - {'CXR1_1_IM-0001-3001': ['startseq the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. endseq'],
 'CXR10_IM-0002-1001': ['startseq the cardiomediastinal silhouette is within normal limits for size and contour. the lungs are normally inflated without evidence of focal airspace disease pleural effusion or pneumothorax. stable calcified granuloma within the right upper lung. no acute bone abnormality. . endseq']
                            }
                            
            max_length = max_length(train_descriptions)
            print('Description Length: %d' % max_length)
            
        - Expected Success Response::

            Description Length: 172
    
    """
    lines = to_lines(descriptions)
    return max(len(d.split('. ')) for d in lines)

# determine the maximum sequence length
max_length = max_length(cleaned_train_descr)
print('Description Length: %d' % max_length)

Description Length: 20


In [45]:
for key in cleaned_train_descr.keys():
    train_dsc=cleaned_train_descr[key][0].split('. ')
    if len(train_dsc)==max_length:
        print(key+' -> ')
        for dscr in train_dsc:
            print(dscr)

CXR3278_IM-1559-0001.png -> 
startseq
chest 2 images
heart size is normal
mediastinal contours are maintained
there is a mild pectus excavatum deformity
the lungs are clear of focal infiltrate
there is no evidence for pleural effusion or pneumothorax
no convincing acute bony findings
right shoulder 3 images
there has been xxxx and screw fixation of the midshaft right clavicle
the lateral most screw is fractured
this is age indeterminate as no prior studies are available for comparison
otherwise the surgical xxxx appears intact
the humeral head is seen within the glenoid without evidence for dislocation
no bony fractures are seen
the visualized right ribs appear intact
right clavicle 2 images
no clavicle fracture is seen
once again noted is the surgical fixation xxxx with fracture of the lateral most fixation screw
endseq


In [46]:
all_Indication_desc = list()
for key in train_indications.keys():
    for d in train_indications[key]:
        all_Indication_desc.append(d)

In [47]:
max_Indication_length = max(len(d.split()) for d in all_Indication_desc)
max_Indication_length

40

### Data generator for image features, indications & report sentences for training

In [48]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, indications, senttoix, indicationWordtoix, max_Indication_length,max_length, num_photos_per_batch):
    """
        **Prepares the input and output data for trainning the model.**
        
        This function will Prepare the input and output data for trainning the model
        
        :param descriptions: image to report key-value pairs
        :type descriptions: dict
        :param photos: Encoded trainning image
        :type photos: array
        :param indications: Indication from trainning report
        :type indications: dict
        :param wordtoix: word to index for findings
        :type indications: list
        :param indicationWordtoix: word to index for indication
        :type indications: list
        :param max_Indication_length: maximum lenght of indication
        :type max_Indication_length: int
        :param max_length: maximum lenght of findings
        :type max_length: int
        :param num_photos_per_batch: number of photos for batch processing
        :type num_photos_per_batch: int
        :return: Input 1, Input 2, Input 3 and output
        :type X1, X2, X3, y: array
        
        - Example::

            generator = data_generator(train_descriptions, train_features,train_indications, wordtoix, indicationWordtoix, max_Indication_length,max_length, number_pics_per_bath)
            newModel.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
                
    """
    X1, X2, X3, y = list(), list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key].squeeze()
            for indication in indications[key]:
                indicationSeq = [indicationWordtoix[word] for word in indication.split(' ') if word in indicationWordtoix]
            indicationSeq = pad_sequences([indicationSeq], maxlen=max_Indication_length)[0]
            for desc in desc_list:
                # encode the sequence
                seq = [senttoix[sent] for sent in desc.split('. ') if sent in senttoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=sent_vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(indicationSeq)
                    X3.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2), array(X3)], array(y)]
                X1, X2, X3, y = list(), list(), list(), list()
                n=0

In [49]:
sentence_embeddings = load(open("KalyanPickle/sent_embeddings.pickle","rb"))

In [50]:
sentence_embeddings.keys()

dict_keys(['', 'cardiomediastinal silhouette appears within normal limits', 'no pneumothoraces', 'stable unfolding the thoracic aorta', 'clear lungs', 'cannot exclude early pulmonary edema', 'there is focal airspace disease in the right lung base concerning for pneumonia or aspiration', 'no free intraperitoneal air in the diaphragm', 'heart is normal size', 'there is stable right upper lung lucency', 'the lungs appear hyperexpanded consistent with emphysema', 'xxxx right perihilar or midlung density', 'scattered xxxx opacities in the right lung base xxxx representing foci of subsegmental atelectasis with scattered airspace opacities in the medial left lower lobe', 'there is scattered areas of bronchial wall thickening well seen in the left upper lobe', 'ct scan may be informative if patient is at high xxxx for lung cancer', 'there is no focal consolidation pneumothorax or large pleural effusion', 'there is also xxxx patchy opacification identified in the left upper lobe', 'stable chron

In [51]:
print(type(sentence_embeddings['there is stable xxxx tube']))
print('Found %s word vectors.' % len(sentence_embeddings))

<class 'numpy.ndarray'>
Found 6325 word vectors.


### Creating embedding matrices for report sentences and indication words

In [52]:
sent_embedding_dim = 700

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((sent_vocab_size, sent_embedding_dim))

for sent, i in senttoix.items():
    #if i < max_words:
    embedding_vector = sentence_embeddings.get(sent)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [53]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('WordEmbeddings/BioWordVec_PubMed_MIMICIII_d200.vec.bin', binary=True)

In [54]:
embeddings_index = {} # empty dictionary
i = 0
for item in word_vectors.vocab:
    word = item
    coefs = np.asarray(word_vectors.get_vector(word), dtype='float32')
    embeddings_index[word] = coefs
    i+=1
print('Found %s word vectors.' % len(embeddings_index))

Found 16545452 word vectors.


In [55]:
indication_embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
indication_embedding_matrix = np.zeros((IndicationVocab_size, indication_embedding_dim))

for word, i in indicationWordtoix.items():
    #if i < max_words:
    indicationembedding_vector = embeddings_index.get(word)
    if indicationembedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        indication_embedding_matrix[i] = indicationembedding_vector

In [56]:
embedding_matrix.shape

(4629, 700)

In [57]:
indication_embedding_matrix.shape

(1292, 200)

### Designing of model for training

In [58]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [59]:
IndicationVocab_size

1292

In [60]:
imageFeature = Input(shape=(1024,))
indication = Input(shape=(max_Indication_length,))
indication1 = Embedding(IndicationVocab_size, indication_embedding_dim)(indication)
indication2 = Flatten()(indication1)
imgIndi = keras.layers.concatenate([imageFeature, indication2],axis=1)
fe1 = Dropout(0.5)(imgIndi)
fe2 = Dense(256, activation='relu')(fe1)
findings = Input(shape=(max_length,))
se1 = Embedding(sent_vocab_size, sent_embedding_dim, mask_zero=True)(findings)
se2 = Dropout(0.5)(se1)
se3 = Bidirectional(LSTM(256, return_sequences=True))(se2)
se4 = LSTM(256)(se3)
decoder1 = add([fe2, se4])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(sent_vocab_size, activation='softmax')(decoder2)
newModel = Model(inputs=[imageFeature, indication, findings], outputs=outputs)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [61]:
newModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 40, 200)      258400      input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
flatten_1 

In [62]:
newModel.layers[1]

In [63]:
newModel.layers[5]

In [64]:
newModel.layers[1].set_weights([indication_embedding_matrix])
newModel.layers[1].trainable = False

In [65]:
newModel.layers[5].set_weights([embedding_matrix])
newModel.layers[5].trainable = False

In [66]:
newModel.compile(loss='categorical_crossentropy', optimizer='adam')

In [67]:
newModel.optimizer.lr = 0.001
epochs = 20
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [68]:
newModel.load_weights('Sent_Embedding_Model_Weights/model_sent_emb_50.h5')

### Greedy Search Approach, Predictions & BLEU score calculations

In [69]:
def greedySearch(photo,indication):
    """
        **Provides the generated text report for a given image and indication**
        
        This function take image and indication as input and sequencialy constructs the text output by calling
        predict on the model. The model returns the conditional probablities of all the words in vocabulary and 
        we pic the word with maximum probablity and append it to the input sequence. This sequence is again presented 
        to the model for predicting the next word.
        
        :param photo: input image
        :type photo: array
        :param indication: input image
        :type indication: array
        :return: Predicted text sentences.
        :type: string
        
        - Example::

            print("Greedy:",greedySearch(image,ind))
            
        - Expected Success Response::

            the lungs are clear. there is no focal airspace consolidation. no pleural effusion or pneumothorax. 
            the heart and mediastinum are within normal limits. xxxx are unremarkable.
    
    """
    in_text='startseq'
    for i in range(max_length):
        sequence = [senttoix[s] for s in in_text.split('. ') if s in senttoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        ind_sequence = [indicationWordtoix[w] for w in indication.split() if w in indicationWordtoix]
        ind_sequence = pad_sequences([ind_sequence], maxlen=max_Indication_length)        
        yhat = newModel.predict([photo,ind_sequence,sequence], verbose=0)
        yhat = np.argmax(yhat)
        sent = ixtosent[yhat]
        in_text += '. ' + sent
        if sent == 'endseq':
            break
    final = in_text.split('. ')
    final = final[1:-1]
    final = '. '.join(final)
    return final

In [74]:
validation_dataset = list()
for img in encoding_test.keys():
    validation_dataset.append(img)

In [75]:
# descriptions
validation_descriptions = load_clean_descriptions(first_sent, validation_dataset)
print('Descriptions: validation=%d' % len(validation_descriptions))

Descriptions: validation=746


In [76]:
# descriptions
validation_indications = load_clean_descriptions(indication_sent, validation_dataset)
print('Descriptions: indication=%d' % len(validation_indications))

Descriptions: indication=746


In [81]:
from nltk.translate.bleu_score import sentence_bleu

In [82]:
def CalculateBLUEScore(predicted, Reference, weights=(1,0,0,0)):
    """
        **Calculates BLEU score for comparing the predicted sentence to the reference sentence**
        
        This function takes the candidate sentence as list of words and compares it with reference 
        list of sentences(list of words). The weights can be altered to configure for matching n-grams of words.
        
        :predicted: candidate/predicted sentence
        :reference: reference sentence
        :weights: weight for nth gram of 4 grams(default 1-gram)
    """
    pred_sents = predicted.split('.')
    actual_sents = Reference.split('.')
    pred_final = []
    actual_final = []
    pred_scores = []

    for sent in pred_sents:
        cur_list = sent.split(' ')
        cur_list = [word for word in cur_list if word]
        pred_final.append(cur_list)
        pred_final = [item for item in pred_final if item]
  
    for sent in actual_sents:
        cur_list = sent.split(' ')
        cur_list = [word for word in cur_list if word]
        actual_final.append(cur_list)
        actual_final = [item for item in actual_final if item]

    for pred_sent in pred_final:
        pred_scores.append(sentence_bleu(actual_final, pred_sent, weights))
        avgScore = sum(pred_scores)/len(pred_scores)
    return avgScore

### General approach for calculation of bleu scores

In [90]:
def get_val_score(pic, weights=(0.25,0.25,0.25,0.25)):
    """
        **For any picture, calculating bleu score of the predicted sentence**
        
        using greedy search for generating report and calculating BLEU score 
        for comparing reference and predicted sentences. This is the general 
        approach for calculating bleu score.
        
        :pic: image encoding
        :weights: gram weights
    """
    image = encoding_test[pic].reshape((1,1024))
    candidate = greedySearch(image,validation_indications[pic][0]).split()
    reference = [validation_descriptions[pic][0].replace('startseq. ','').replace(' endseq','').split()]
    return sentence_bleu(reference,candidate,weights)

In [91]:
def get_val_avg_scores():
    """
        **Calculating average bleu scores for 1-gram, 2-gram, 3-gram and 4-gram**
        
        calling average bleu score calculation for different grams.
    """
    pictures = list(encoding_test.keys())
    total_bleu_4 = 0.0
    total_bleu_3 = 0.0
    total_bleu_2 = 0.0
    total_bleu_1 = 0.0
    cnt = 0
    for pict in pictures:
        if pict in validation_indications.keys():
            total_bleu_4 += get_val_score(pict,(0.0,0.0,0.0,1.0))
            total_bleu_3 += get_val_score(pict,(0.0,0.0,1.0,0.0))
            total_bleu_2 += get_val_score(pict,(0.0,1.0,0.0,0.0))
            total_bleu_1 += get_val_score(pict,(1.0,0.0,0.0,0.0))
            cnt += 1
    return (total_bleu_4/cnt,total_bleu_3/cnt,total_bleu_2/cnt,total_bleu_1/cnt)

In [97]:
(bleu_4,bleu_3,bleu_2,bleu_1)=get_val_avg_scores()

In [98]:
bleu_4

0.035541197581053834

In [99]:
bleu_3

0.04963234748334065

In [100]:
bleu_2

0.08665122742451793

In [101]:
bleu_1

0.23367220601464533

### New Approach for calculating BLEU scores

In [83]:
def get_new_score(pic,weights):
    """
        **For any picture, calculating bleu score of the predicted sentence**
        
        using greedy search for generating report and calculating BLEU score 
        for comparing reference and predicted sentences. This is a new approach
        for calculating bleu score.
        
        :pic: image encoding
        :weights: gram weights
    """
    image = encoding_test[pic].reshape((1,1024))
    candidate = greedySearch(image,validation_indications[pic][0])
    reference = validation_descriptions[pic][0].replace('startseq. ','').replace(' endseq','')
    return CalculateBLUEScore(candidate,reference,weights)

In [84]:
def get_new_avg_scores():
    """
        **Calculating average bleu scores for 1-gram, 2-gram, 3-gram and 4-gram**
        
        calling average bleu score calculation for different grams.
    """
    pictures = list(encoding_test.keys())
    total_bleu_4 = 0.0
    total_bleu_3 = 0.0
    total_bleu_2 = 0.0
    total_bleu_1 = 0.0
    cnt = 0
    for pict in pictures:
        if pict in validation_indications.keys():
            total_bleu_4 += get_new_score(pict,(0.0,0.0,0.0,1.0))
            total_bleu_3 += get_new_score(pict,(0.0,0.0,1.0,0.0))
            total_bleu_2 += get_new_score(pict,(0.0,1.0,0.0,0.0))
            total_bleu_1 += get_new_score(pict,(1.0,0.0,0.0,0.0))
            cnt += 1
    return (total_bleu_4/cnt,total_bleu_3/cnt,total_bleu_2/cnt,total_bleu_1/cnt)

In [85]:
bleu4,bleu3,bleu2,bleu1 = get_new_avg_scores()

D:\Anaconda\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\Anaconda\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\Anaconda\lib\site-packages\nltk\translate\bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [86]:
bleu1

0.3913455310292566

In [87]:
bleu2

0.14127565267553732

In [88]:
bleu3

0.08309531911615366

In [89]:
bleu4

0.05298140732666435

### Beam search for predicting sentences

In [102]:
def beam_search_predictions(pic, indications, beam_index = 3):
    """
        **Provides the generated text report for a given image feature, indication and beam_index**
        
        This function take image feature and indication as input and sequencialy constructs the text output by calling
        predict on the model. The model returns the conditional probablities of all the words in vocabulary and 
        we sort the words based on probablity and pic the group of maximum probable words indicated by beam index, 
        append it to the next input sequence. This sequence is again presented to the model for predicting the next word.
        
        :param photo: input image
        :type photo: array
        :param indication: input image
        :type indication: array
        :param beam_index: indicates how many words has to be grouped
        :type beam_index: int
        :return: Predicted text sentences.
        :type: string
        
        - Example::

            print(beam_search_predictions(image,ind))
            
        - Expected Success Response::

            the lungs are clear. there is no focal airspace consolidation. no pleural effusion or pneumothorax. 
            the heart and mediastinum are within normal limits. xxxx are unremarkable.
    
    """
    start = [senttoix["startseq"]]
    
    # start_word[0][0] = index of the starting word
    # start_word[0][1] = probability of the word predicted
    start_sent = [[start, 0.0]]
    
    while len(start_sent[0][0]) < max_length:
        temp = []
        for s in start_sent:
            par_caps = sequence.pad_sequences([s[0]], maxlen=max_length, padding='post')
            #e = encoding_test[photo[len(images):]]
            #preds = model.predict([np.array([e]), np.array(par_caps)])
            ind_sequence = [indicationWordtoix[w] for w in indications.split() if w in indicationWordtoix]
            ind_sequence = pad_sequences([ind_sequence], maxlen=max_Indication_length)
            preds = newModel.predict([pic,ind_sequence,np.array(par_caps)], verbose=0)
            
            # Getting the top <beam_index>(n) predictions
            sent_preds = np.argsort(preds[0])[-beam_index:]
            
            # creating a new list so as to put them via the model again
            for sp in sent_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(sp)
                prob += preds[0][sp]
                temp.append([next_cap, prob])
                    
        start_sent = temp
        # Sorting according to the probabilities
        start_sent = sorted(start_sent, reverse=False, key=lambda l: l[1])
        # Getting the top words
        start_sent = start_sent[-beam_index:]
    
    start_sent = start_sent[-1][0]
    intermediate_caption = [ixtosent[i] for i in start_sent]

    final_caption = []
    
    for i in intermediate_caption:
        if i != 'endseq':
            final_caption.append(i)
        else:
            break
    
    final_caption = '. '.join(final_caption[1:])
    return final_caption

In [104]:
def get_new_score(pic,weights):
    """
        **For any picture, calculating bleu score of the predicted sentence**
        
        using greedy search for generating report and calculating BLEU score 
        for comparing reference and predicted sentences. This is a new approach
        for calculating bleu score.
        
        :pic: image encoding
        :weights: gram weights
    """
    image = encoding_test[pic].reshape((1,1024))
    candidate = beam_search_predictions(image,validation_indications[pic][0])
    reference = validation_descriptions[pic][0].replace('startseq. ','').replace(' endseq','')
    return CalculateBLUEScore(candidate,reference,weights)

In [105]:
def get_new_avg_scores():
    """
        **Calculating average bleu scores for 1-gram, 2-gram, 3-gram and 4-gram**
        
        calling average bleu score calculation for different grams.
    """
    pictures = list(encoding_test.keys())
    total_bleu_4 = 0.0
    total_bleu_3 = 0.0
    total_bleu_2 = 0.0
    total_bleu_1 = 0.0
    cnt = 0
    for pict in pictures:
        if pict in validation_indications.keys():
            total_bleu_4 += get_new_score(pict,(0.0,0.0,0.0,1.0))
            total_bleu_3 += get_new_score(pict,(0.0,0.0,1.0,0.0))
            total_bleu_2 += get_new_score(pict,(0.0,1.0,0.0,0.0))
            total_bleu_1 += get_new_score(pict,(1.0,0.0,0.0,0.0))
            cnt += 1
    return (total_bleu_4/cnt,total_bleu_3/cnt,total_bleu_2/cnt,total_bleu_1/cnt)

In [107]:
bleu_4,bleu_3,bleu_2,bleu_1 = get_new_avg_scores()

In [111]:
bleu_4

0.05809796385680246

In [110]:
bleu_3

0.09140356392670301

In [109]:
bleu_2

0.15253958538246568

In [108]:
bleu_1

0.404714239680934

### Sample prediction

In [92]:
first_sent['CXR3533_IM-1726-1001']

['startseq heart size mediastinal contour and pulmonary vascularity are within normal limits. there is bilateral hyperinflation without focal consolidation pneumothorax or pleural effusion. visualized osseous structures appear intact. endseq']

In [93]:
ind1=train_indications['CXR3533_IM-1726-1001.png']

In [94]:
ind1

['xxxx year old xxxx with shortness of breath xxxx. .']

In [95]:
greedySearch(train_features['CXR3533_IM-1726-1001.png'],ind1[0])

'the heart size and cardiomediastinal silhouette are normal. pulmonary vascularity is normal. lungs are clear. no focal consolidation pneumothorax or pleural effusion'

In [96]:
sentence_bleu(['heart size mediastinal contour and pulmonary vascularity are within normal limits. there is bilateral hyperinflation without focal consolidation pneumothorax or pleural effusion. visualized osseous structures appear intact.'.split()],'no acute osseous abnormality. the soft tissues are within normal limits. normal appearing cardiomediastinal silhouette and hilar contours. left lower lobe xxxx density xxxx representing atelectasis. no focal area of consolidation pleural effusion pneumothorax'.split())

0.08568635726825899

In [103]:
beam_search_predictions(train_features['CXR3533_IM-1726-1001.png'],ind1[0])

'cardiac and mediastinal contours are unremarkable. pulmonary vascularity is within normal limits. no focal air space opacities pleural effusion or pneumothorax. xxxx are grossly unremarkable'